In [11]:
from keras.layers import TFSMLayer
import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np
img = load_img('test/hat/2a12baab-f020-42e3-8e6b-5d82e3ed0b55.jpg',target_size = (299,299))
x= np.array(img)
X = np.array([x])
X = preprocess_input(X)
model = TFSMLayer("xception_v1_03_0.827", call_endpoint="serving_default")
# Use like:
output = model(X)
output

{'dense_7': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-0.99701434,  2.564598  , -0.6685115 ,  1.2119223 , -1.0608624 ,
         -1.585297  ,  1.8948718 , -0.961256  , -0.664027  ,  0.1305011 ]],
       dtype=float32)>}

In [12]:
import tensorflow as tf

# From saved model folder
converter = tf.lite.TFLiteConverter.from_saved_model("xception_v1_03_0.827")
tflite_model = converter.convert()

# Save as .tflite
with open("clothing_model.tflite", "wb") as f:
    f.write(tflite_model)


W0000 00:00:1752779070.627886   42378 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1752779070.627922   42378 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-17 19:04:30.629200: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: xception_v1_03_0.827
2025-07-17 19:04:30.647080: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-17 19:04:30.647094: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: xception_v1_03_0.827
I0000 00:00:1752779070.727123   42378 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-17 19:04:30.738845: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-17 19:04:31.076905: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: xception_v1_03_0.827
2025-07-17 19:04:31.185390: I tensorflow/cc/saved_model/loader.cc:471]

In [13]:
import tensorflow.lite as tflite

In [14]:
interpreter = tflite.Interpreter(model_path = 'clothing_model.tflite')
interpreter.allocate_tensors()

/usr/local/python/3.12.1/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [22]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
interpreter.set_tensor(input_index,X)
interpreter.invoke()
interpreter.get_tensor(output_index)

array([[-0.9970156 ,  2.5646005 , -0.6685142 ,  1.2119219 , -1.0608639 ,
        -1.5852971 ,  1.894876  , -0.961256  , -0.66402775,  0.13050205]],
      dtype=float32)

In [27]:
classes = ['dress',
 'hat',
 'longsleeve',
 'outwear',
 'pants',
 'shirt',
 'shoes',
 'shorts',
 'skirt',
 't-shirt']
dict(zip(classes,interpreter.get_tensor(output_index)[0]))

{'dress': np.float32(-0.9970156),
 'hat': np.float32(2.5646005),
 'longsleeve': np.float32(-0.6685142),
 'outwear': np.float32(1.2119219),
 'pants': np.float32(-1.0608639),
 'shirt': np.float32(-1.5852971),
 'shoes': np.float32(1.894876),
 'shorts': np.float32(-0.961256),
 'skirt': np.float32(-0.66402775),
 't-shirt': np.float32(0.13050205)}

### REmoving TF dependency

In [7]:
!pip install ai-edge-litert

In [8]:
from PIL import Image
import numpy as np
from ai_edge_litert.interpreter import Interpreter
#import tensorflow.lite as tflite
hat_url = 'test/hat/2a12baab-f020-42e3-8e6b-5d82e3ed0b55.jpg'
with Image.open(hat_url) as img:
    img = img.resize((299,299),Image.NEAREST)
x= np.array(img, dtype = 'float32')
X = np.array([x])

In [9]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [10]:
X= preprocess_input(X)

In [11]:
interpreter = Interpreter(model_path = 'clothing_model.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
interpreter.set_tensor(input_index,X)
interpreter.invoke()
output = interpreter.get_tensor(output_index)
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
#dict(zip(classes,interpreter.get_tensor(output_index)[0]))
# Find the index with the highest score
pred_index = np.argmax(output[0])
# Map to class
predicted_class = classes[pred_index].capitalize()
predicted_score = output[0][pred_index]

# Display
print(f"Predicted class: {predicted_class} (score: {predicted_score:.4f})")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Predicted class: Hat (score: 2.5646)
